# Rapport du projet de programmation

## 1. Introduction

Le projet de programmation que nous avons entrepris se veut proposer des formes d'analyses et de réponses au sujet des thématiques et problématiques qui touchent l'utilisation des réseaux sociaux à des fins d'information, notamment Twitter. En effet, la construction de véritables empires médiatiques est croissante et tend à se normaliser. Le choix de s'intéresser directement à ce qui se passe sur un des réseaux sociaux les plus utilisés au monde nous a donc semblé pertinent dans la mesure où Twitter est sujet à de multiples controverses. **comment ça se fait que quand tu viens de créer un compte tu tombes sur une page de trump ou de elon musk fin bon pas à moi (en vrai jsuis chaud qu'on en parle)** Dans les événements récents, de nombreuses personnalités ont notamment appelé à quitter Twitter (de grands journaux tels que le *Guardian*, des journalistes ainsi que des personnalités politiques). Mais qu'en est-il des utilisateurs ? Quelle attitude adoptent-ils face à cela ?

Les enjeux de ce projet portent ainsi concrètement sur le niveau de prises de position, qu'elles aillent dans un sens favorable ou défavorable à ce que nous disons être une main mise d'Elon Musk sur Twitter. De fait, ce projet interroge aussi une forme d'éveil citoyen et politique des utilisateurs face aux informations qu'ils reçoivent, mais aussi le niveau d'implication et de mobilisation dans ce type de discussions.
Ainsi, ce projet suit une ligne directrice orientée, car nous partons effectivement du principe que les utilisateurs se posent la question et réagissent à la mouvance qui consiste à quitter Twitter (nous-même partons du fait que nous identifions un problème de désinformation sur Twitter).

**présentation du plan d'action**
Concrètement, nous avons décidé de scrapper Twitter pour obtenir une base de données qui remonte jusqu'à.... : nous avons ainsi collecté le nom d'utilisateur, la date, le contenu du tweet, le nombre de likes, de réponses, de retweets et de vues.

Pour analyser les tweets, nous avons mis en place du NLP pour tenter d'obtenir une information supplémentaire sur les contenus des tweets, et ne pas seulement s'en tenir à des informations très descriptives reflétées par le nombre de likes, réponses, retweets et vues.

on apprend à thibault qui attend 

On commence par installer les différentes bibliothèques qui seront nécessaires à l'exécution des codes :

In [2]:
!pip install pandas
!pip install matplotlib
!pip install datetime
!pip install openpyxl
!pip install selenium

## 2. Base de données

### 2.1 Scrapping
Une fois ceci fait, la première étape va consister à scrapper Twitter. La façon dont nous avons procédé a été la suivante :

- Tout d'abord nous avons installé le framework Selenium, afin de naviguer automatiquement sur le site de Twitter et faire défiler les tweets sur la page qui nous intéressait. En effet nous souhaitions récolter les tweets dans leur ordre de publication, c'est-à-dire dans l'onglet "Récents" de la page de recherche de Twitter. 
- Pour cela, nous avons créé un compte Twitter : l'utilisateur et le mot de passe sont remplis lors de l'exécution du code. Dans la barre de recherche nous avons donc entré les mots clés que nous voulions voir ressortir dans les tweets (les mots liés au fait de quitter Twitter et le mot "Musk"), de janvier 2024 à décembre 2024. 
- Afin de récupérer les tweets et les stocker dans un dataframe, nous avons identifié l'auteur, le contenu, la date, le nombre de vues, de likes, de reposts et de commentaires de chaque tweet en trouvant l'emplacement de ces éléments dans la structure HTML de la page web, puis avons placé chacun d'eux dans une liste. Cela nous a permis d'obtenir un tableau mis sous format Excel.

**ATTENTION : Le code met du temps à tourner, nous avons donc fourni directement les bases de données dont nous nous servons sous format Excel**

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd

driver = webdriver.Chrome()
driver.get('https://x.com/i/flow/login')
time.sleep(10)

# Connexion
username_input = driver.find_element('name', 'text')
username_input.send_keys('philipa.seilles@ensae.fr')
username_input.send_keys(Keys.RETURN)
time.sleep(5)

webdriver.ActionChains(driver).send_keys('car_pelet').perform()
webdriver.ActionChains(driver).send_keys(Keys.RETURN).perform()
time.sleep(5)

password_input = driver.find_element('name', 'password')
password_input.send_keys('w7a4731O')
password_input.send_keys(Keys.RETURN)
time.sleep(5)

# Navigue vers la recherche
explore_button = driver.find_element(By.XPATH, "//a[@href='/explore']")
explore_button.click()
time.sleep(5)

search_bar = driver.find_element(By.XPATH, "//input[@aria-label='Search query']")
search_bar.send_keys("(leaving OR leave OR quitting) twitter musk until:2024-12-03 since:2024-01-01")
search_bar.send_keys(Keys.RETURN)
time.sleep(7)

link = driver.find_element(By.XPATH, "(//div[@class='css-175oi2r r-18u37iz r-16y2uox r-1wbh5a2 r-tzz3ar r-1pi2tsx r-buy8e9 r-mfh4gg r-2eszeu r-10m9thr r-lltvgl']//a)[2]")
link.click()
time.sleep(6)

# Listes pour stocker les données des tweets
usernames = []
dates = []
contents = []
comments = []
repost = []
likes = []
views = []

# Ensemble pour vérifier l'unicité des tweets
tweets_seen = set()


# Fonction pour collecter des tweets
def collect_tweets():
    new_tweets_found = False 
    tweet_elements = driver.find_elements(By.XPATH, "(//div[contains(@class, 'css-175oi2r r-1igl3o0 r-qklmqi r-1adg3ll r-1ny4l3l')])")
    for element in tweet_elements:
        try:
            username_recup = element.find_element(By.XPATH, ".//a[contains(@class, 'css-175oi2r r-1wbh5a2 r-dnmrzs r-1ny4l3l r-1loqt21')]").text
            date_recup = element.find_element(By.XPATH, ".//div[contains(@class, 'css-175oi2r r-18u37iz r-1q142lx')]").text
            content_recup = element.find_element(By.XPATH, ".//div[contains(@class, 'css-146c3p1 r-8akbws r-krxsd3 r-dnmrzs r-1udh08x r-bcqeeo r-1ttztb7 r-qvutc0 r-37j5jr r-a023e6 r-rjixqe r-16dba41 r-bnwqim')]").text
            stats_recup = element.find_elements(By.XPATH, ".//div[contains(@class, 'css-175oi2r r-xoduu5 r-1udh08x')]")
            comments_recup = stats_recup[0].text
            repost_recup = stats_recup[1].text
            likes_recup = stats_recup[2].text
            views_recup = stats_recup[3].text
            tweet_tuple = (username_recup, date_recup, content_recup)
            
            # Vérifier si le tweet a déjà été vu
            if tweet_tuple not in tweets_seen:
                tweets_seen.add(tweet_tuple) 
                usernames.append(username_recup)
                dates.append(date_recup)
                contents.append(content_recup)
                comments.append(comments_recup)
                repost.append(repost_recup)
                likes.append(likes_recup)
                views.append(views_recup)
                new_tweets_found = True  # Détecter un nouveau tweet

        except Exception as e:
            continue
    return new_tweets_found  # Retourne True si de nouveaux tweets ont été ajoutés


# Boucle pour défiler et collecter jusqu'à obtenir 10 000 tweets
max_no_new_tweets = 5  # Nombre maximal de tentatives sans nouveaux tweets avant de s'arrêter
no_new_tweets_count = 0  # Compteur pour les tentatives sans nouveaux tweets

while len(usernames) < 10000 and no_new_tweets_count < max_no_new_tweets:
    if collect_tweets(): 
        no_new_tweets_count = 0  # Réinitialiser le compteur si des tweets ont été collectés
    else:
        no_new_tweets_count += 1  # Si aucun tweet n'est collecté, incrémenter le compteur

    # Scrolling pour charger davantage de tweets si aucun nouveau tweet n'est trouvé
    for _ in range(5):  # Défiler 5 fois
        driver.execute_script("window.scrollBy(0, window.innerHeight);")  # Défiler de la hauteur de la fenêtre
        time.sleep(1)  # Attendre le chargement des nouveaux tweets        
        # Récupérer des nouveaux tweets après chaque défilement
        if collect_tweets():  # Si de nouveaux tweets sont trouvés après le défilement
            no_new_tweets_count = 0  # Réinitialiser le compteur
            break  # Sort de la boucle si de nouveaux tweets sont trouvés

    if no_new_tweets_count >= max_no_new_tweets:  # Si le nombre de tentatives sans nouveaux tweets est trop élevé
        print("Aucun nouveau tweet trouvé après plusieurs tentatives, arrêt.")
        break

print(f"Total des tweets récupérés : {len(usernames)}")
driver.quit()

# Créer un DataFrame et enregistrer les données dans un fichier Excel
data = {
    'Username': usernames,
    'Date': dates,
    'Content': contents,
    'Comments': comments,
    'Repost': repost,
    'Likes': likes,
    'Views': views
}

df = pd.DataFrame(data)
df.to_excel("tweets_leave.xlsx", index=False)
print("Fichier tweets_leave.xlsx créé avec succès.")

Nous exécutons le code plusieurs fois en changeant les mots clefs : **rajouter les mots clefs**:
- quit twitter musk
- leave or leaving toussa
- remain blabla

Nous allons garder plusieurs tables :
- une table avec les mots clefs "quit" et "leave" : nous gardons ici de façon générale les personnes qui disent quitter Twitter
- une table qui regroupe les personnes qui disent rester sur Twitter (mots clefs "remain on", "still on")
- une table qui regroupe ces deux tables, qui réunit donc les personnes qui *s'expriment* sur le sujet, et qui se posent donc la question de rester ou non sur Twitter.

Nous ajoutons à cela une phase de nettoyage de ces tables : 
- pour regrouper les tables, nous les ajoutons les unes aux autres, convertissons le tout dans un format date qui permet ensuite de trier et de réarranger la table dans le bon ordre temporel.
- les cases vides susceptibles d'apparaître dans les colonnes "Likes", "Views", etc... sont remplacées par des 0 (il n'y a eu en effet aucun like par exemple).
 **Nettoyer la table ? quelles dates on garde ? mettre des 0 là où y a des trous => mieux si philipa s'en charge?**. Nous avons ainsi en colonnes blablabla. **présenter une sortie du tableau excel ?**

** A FAIRE, LUCIE A LES CODES**
+ code pour regrouper quit et not leave

+ convertir tout en format date
Les dates que nous obtenons directement après avoir scrappé ne sont pas dans un format "date" qui simplifie les calculs. Nous utilisons donc le module *datetime* de Python, notamment la fonction *datetime.strptime* pour convertir une chaîne de caractères représentant une date ou une heure en un objet datetime. En effet, nous avons été particulièrement vigilants au fait que notre code devait faire en sorte d'assigner la bonne date aux dates apparaissant sous le format "6mn", "1h" : dans notre code, nous faisons en sorte de prendre en compte la date suivant la précédente. Ainsi, nous avons pu régler des problèmes de date d'exécution du code.

+ trier 

+ cases vides remplacées par des 0

PAS OUBLIER DE METTRE LES ANCIENNES (celles avec le scrapping) ET LES NOUVELLES TABLES (nettoyées et belles)

Ci-dessous nous présentons une sortie de notre table **mettre le nom de la table (celle avec tout)** qui affiche les 10 premières lignes (en affichant tout le contenu, ce que permet *display*) de notre table :

In [1]:
import pandas as pd

# Modifier les paramètres pour afficher plus de contenu dans chaque cellule
pd.set_option('display.max_colwidth', None)  # Affiche tout le contenu des colonnes
pd.set_option('display.max_columns', None)  # Affiche toutes les colonnes
pd.set_option('display.width', 1000)  # Ajuste la largeur totale de l'affichage

# Charger et afficher le fichier Excel
df = pd.read_excel('tweets_fusionnes.xlsx')
df.head(10)


,Username,Date,Content,Comments,Repost,Likes,Views,ConvertedDate,YearWeek,YearMonth
0,𝐇𝐎𝐓𝐄𝐏 悪いバナー,Nov 30,"The left have become so dependent on Twitter that they're forced with two options:\n\n1) Leave Twitter and leave behind the one platform that they feel grants them so much power.\n\n2) Stay on Twitter but deal with Musk, MAGA, Trump, etc.",1,0,0,49,2024-11-30,2024-47,2024-11
1,NYCGRIFF,Nov 30,"I noticed. I'm about ready to leave this Twitter dance. Bluesky is a much better place to hangout. Still needs improvement in some key areas. But, this joint is definitely getting out of hand with Musk and his crowd doing a lot of funky shit. Plus, people are jumping ship L & R.",0,0,1,30,2024-11-30,2024-47,2024-11
2,StraitjacketEnjoyer,Nov 30,Have not noticed anything out of the ordinary since Musk bought Twitter.\n\nSome people are annoying but you can always just block them.\n\nAny specific reason you will leave Twitter or is it just that you don't like the platform owner?,0,0,0,47,2024-11-30,2024-47,2024-11
3,"Laura Starrett, afide",Nov 30,Will be following you there. I originally planned to leave Twitter on January 20 but with Musk’s latest threats and attempts to be king I changed it to Dec. 31. Might not make it until then.,0,0,2,23,2024-11-30,2024-47,2024-11
4,Dennis C. Hayes,Nov 30,"We cannot leave him to demonstrate that he can take the bias out of the meta-verse. It’s built into every element of the system. Look at what Elon Musk discovered when he took the covers off of Twitter. The bias was built into every level of the system in this operation, the same",0,0,2,53,2024-11-30,2024-47,2024-11
5,Marie Robinson,Nov 30,I'm leaving X (formally Twitter) and going to BlueSky. I will not support anything connected to Elon Musk!!,0,0,1,27,2024-11-30,2024-47,2024-11
6,Adrienne Bradley,Nov 30,Change to Twitter Suggests Elon Musk Is Panicking Over Users Leaving for Bluesky,0,0,1,24,2024-11-30,2024-47,2024-11
7,Holly,Nov 30,"You suck worse. people are leaving twitter for BlueSky, and increasingly so, like Holly will soon do, because as the chill people leave, all that's left is the so over-caring people, with your grievances, like Elon Musk.",1,0,1,48,2024-11-30,2024-47,2024-11
8,Dan Travers,Nov 30,"Most liberal democrats are Dictator Marxist who don’t want Free Speech because they think it offends people. \n\nThey left Twitter because NOW Elon Musk a TRUE Patriot, has been ALLOWING FREE SPEECH back on Twitter then when Dorsey had it.\n\nThat’s GOOD you’re not leaving David.",0,0,0,19,2024-11-30,2024-47,2024-11
9,Deb's dizzy,Nov 30,Change to Twitter Suggests Elon Musk Is Panicking Over Users Leaving for Bluesky,1,0,0,39,2024-11-30,2024-47,2024-11


Voilà à quoi ressemble notre sortie : l'étape d'après est le NLP.

### 2.2 NLP

partie de thibault

## Visualisation
Une fois la base de données nettoyée après avoir été obtenue par scrapping, l'objectif est de l'organiser et de créer les variables nécessaires à la création d'indices, eux-mêmes nécessaires à la réalisation de statistiques descriptives et d'analyses.

### Rajout d'une colonne YearWeek et YearMonth
Nous avons choisi d'analyser nos données en les groupant par semaine et par mois, selon les observations que nous voulons faire, ce qui permet d'avoir une analyse assez fine des tendances sans être trop exposés aux fluctuations journalières.

Dans le code suivant, nous avons créé une nouvelle table avec un rajout des colonnes Yearweek et YearMonth. 


code 0_rap_convertdate

## Premiers graphiques
Une fois ceci fait, nous pouvons sortir nos premiers graphiques :
- d'abord le **nombre de tweets par semaine**, ce qui permet de relever certaines tendances
- puis sur un autre graphique nous représentons le **nombre de likes, de retweets et de réponses par semaine** en **valeur absolue** puis en **valeur pondérée** par le nombre de tweets.

In [ ]:
code 1_rap_nbrtweets
code 2_rap_nbrlikes

CONCLUSION de ces premiers éléments


- ensuite, nous avons décidé de **créer différents indices** pour aborder autrement les données. 
1. un taux d'engagement égal à la somme de likes, de retweets, de réponses le tout sur le nombre de vues, exprimé en pourcentages. Nous mettons en place un taux d'engagement par semaine. 
2. un taux de croissance des interactions : nous nous sommes ici restreints à la variation de likes d'une semaine à l'autre
**PONDERER PAR LE NOMBRE DE TWEETS**

In [ ]:
code 3_rap_indices

Une fois ces indices créés, nous les représentons sur un même graphique :

In [ ]:
4_graph_indices